In [1]:
import platform, os, sys, datetime, re
from os.path import join
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
# sys.path.append(join(root_dir,'cvtracer'))
from cvt.TrAQ.Trial import Trial
from cvt.TrAQ.Tank import Tank
from cvt.TrAQ.Group import Group
from cvt.TrAQ.CVTracer import CVTracer, create_named_window, wait_on_named_window

In [4]:
trial_files = sorted(glob('../tracking/full/*/trial.pik'))
display(trial_files)

['../tracking/full/Pa_Fri_14dpf_GroupA_n2_15FPS/trial.pik',
 '../tracking/full/Pa_Fri_14dpf_GroupA_n2b_15FPS/trial.pik',
 '../tracking/full/Pa_Fri_14dpf_GroupA_n2b_30FPS-0000/trial.pik',
 '../tracking/full/Pa_Fri_14dpf_GroupB_n2_-0000-0000/trial.pik',
 '../tracking/full/Pa_Fri_14dpf_GroupB_n2b_0000/trial.pik',
 '../tracking/full/Pa_Fri_14dpf_GroupC_n2_-0000-0000/trial.pik',
 '../tracking/full/Pa_Fri_14dpf_GroupD_n2_-0000-0000/trial.pik',
 '../tracking/full/Pa_Fri_14dpf_GroupD_n2b_-0000/trial.pik',
 '../tracking/full/Pa_Fri_7dpf_GroupA_n2_2020-06-05-120920-0000/trial.pik',
 '../tracking/full/Pa_Fri_7dpf_GroupA_n2b_2020-06-05-103456-0000/trial.pik',
 '../tracking/full/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000/trial.pik',
 '../tracking/full/Pa_Fri_7dpf_GroupB_n2b_2020-06-05-114635-0000/trial.pik',
 '../tracking/full/Pa_Fri_7dpf_GroupB_n5_2020-06-05-094643-0000/trial.pik',
 '../tracking/full/SF_Sat_14dpf_GroupA_n2a_2020-06-13-090438-0000/trial.pik',
 '../tracking/full/SF_Sat_14dpf_Group

In [5]:
for trial_file in trial_files:

    trial = Trial()
    trial.load(trial_file)
    trial.convert_pixels_to_cm()
    trial.calculate_kinematics()
    # trial.save()

    val_name  = ['dwall', 'speed', 'omega']
    val_range = [ [0, trial.tank.r_cm], 
                  [0, trial.tank.r_cm], 
                  [-20, 20] ]
    val_bins  = [ 40, 40, 40 ]
    n_buffer_frames = 2

    tag = trial.evaluate_cuts(n_buffer_frames = n_buffer_frames,
                              ocut = 0,
                              vcut = val_range[1],
                              wcut = val_range[2] )

    trial.calculate_statistics(val_name = val_name, 
                               val_range = val_range, 
                               val_bins = val_bins,
                               ocut = True, vcut = True, wcut = True,
                               tag = tag)
    
    # Plot the joint distribution of pair distance and pair angle.
    trial.group.collect_distance_alignment(ocut = True, 
                                           vcut = True, 
                                           wcut = True)
    dist_align = np.copy(trial.group.dij_mij)
    dist_align[:,1] = 180/np.pi*np.arccos(dist_align[:,1]) # convert to degrees
    plt.hist2d(dist_align[:,0], dist_align[:,1], bins = [20,20], 
               range=[[0,2*trial.tank.r_cm],[0,180]], cmap=plt.cm.hot_r)
    plt.xlabel("Pair distance (cm)")
    plt.ylabel("Pair angle (deg)")
    plt.colorbar()
    plt.savefig(trial.make_fig_file('pairs',tag),**trial.plot_options)
    plt.clf()
    
    # Plot the angle vs time for each fish.
    for f in trial.group.fish:
        t   = f.df['t']
        th  = f.df['theta']
        dth = th.diff()
        dth = dth - 2*np.pi*np.rint(dth/(2*np.pi))
        th  = np.cumsum(dth)
        plt.plot(t,th) #,marker='.')
    plt.xlabel('Time (s)')
    plt.ylabel('Angle (rad)')
    plt.savefig(trial.make_fig_file('angle-vs-time',''),**trial.plot_options)    
    plt.clf()
    


        Trial loaded from ../tracking/full/Pa_Fri_14dpf_GroupA_n2_15FPS/trial.pik 

       Converting pixels to (x,y) space in (cm,cm).

       using to tank size and location from ../tracking/full/Pa_Fri_14dpf_GroupA_n2_15FPS/tank.pik
       Calculating kinematics...

        Trial object saved as ../tracking/full/Pa_Fri_14dpf_GroupA_n2_15FPS/trial.pik 

       ... kinematics calculated for Trial and saved in
             ../tracking/full/Pa_Fri_14dpf_GroupA_n2_15FPS/trial.pik 


  Generating occlusion cut using min d_nn = 0.00 cm with 2 buffer frames
 Shape before cuts:  (34663, 2)
 Shape after cuts:  (34663, 2)
    ... for fish 0, 
 Shape before cuts:  (34663, 2)
 Shape after cuts:  (34663, 2)
    ... for fish 1, 


  Generating speed cut using range [0.00, 0.52] cm/s with 2 buffer frames
    ... for fish 0, 
    ... for fish 1, 


  Generating angular speed cut using range [-20.00, 20.00] rad/s with 2 buffer frames
    ... for fish 0, 
    ... for fish 1, 


  Combining all cuts..

 Shape before cuts:  (44616, 2)
 Shape after cuts:  (35313, 2)
 Shape before cuts:  (44616, 2)
 Shape after cuts:  (34233, 2)

        Trial loaded from ../tracking/full/Pa_Fri_14dpf_GroupC_n2_-0000-0000/trial.pik 

       Converting pixels to (x,y) space in (cm,cm).

       using to tank size and location from ../tracking/full/Pa_Fri_14dpf_GroupC_n2_-0000-0000/tank.pik
       Calculating kinematics...

        Trial object saved as ../tracking/full/Pa_Fri_14dpf_GroupC_n2_-0000-0000/trial.pik 

       ... kinematics calculated for Trial and saved in
             ../tracking/full/Pa_Fri_14dpf_GroupC_n2_-0000-0000/trial.pik 


  Generating occlusion cut using min d_nn = 0.00 cm with 2 buffer frames
 Shape before cuts:  (38054, 2)
 Shape after cuts:  (38054, 2)
    ... for fish 0, 
 Shape before cuts:  (38054, 2)
 Shape after cuts:  (38054, 2)
    ... for fish 1, 


  Generating speed cut using range [0.00, 0.52] cm/s with 2 buffer frames
    ... for fish 0, 
    ... for fish 1, 


  Gene

 mean and err 5.766024729769658e-21 2.7911734122703975e-19
 mean and err 0.7355732735758204 0.06071092972398024
 mean and err 144.87999030080664 27.572110635563618
 Shape before cuts:  (38143, 2)
 Shape after cuts:  (36536, 2)
 Shape before cuts:  (38143, 2)
 Shape after cuts:  (35484, 2)

        Trial loaded from ../tracking/full/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000/trial.pik 

       Converting pixels to (x,y) space in (cm,cm).

       using to tank size and location from ../tracking/full/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000/tank.pik
       Calculating kinematics...

        Trial object saved as ../tracking/full/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000/trial.pik 

       ... kinematics calculated for Trial and saved in
             ../tracking/full/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000/trial.pik 


  Generating occlusion cut using min d_nn = 0.00 cm with 2 buffer frames
 Shape before cuts:  (37636, 2)
 Shape after cuts:  (37636, 2)
    ... for fish 0, 
 S

 Shape after cuts:  (33004, 2)
 Shape before cuts:  (37359, 2)
 Shape after cuts:  (33004, 2)
 Shape before cuts:  (37359, 2)
 Shape after cuts:  (35908, 2)
 Shape before cuts:  (37359, 2)
 Shape after cuts:  (35908, 2)
 Shape before cuts:  (37359, 2)
 Shape after cuts:  (35908, 2)
 Shape before cuts:  (37359, 2)
 Shape after cuts:  (35908, 2)

        Trial loaded from ../tracking/full/SF_Sat_14dpf_GroupA_n2a_2020-06-13-090438-0000/trial.pik 

       Converting pixels to (x,y) space in (cm,cm).

       using to tank size and location from ../tracking/full/SF_Sat_14dpf_GroupA_n2a_2020-06-13-090438-0000/tank.pik
       Calculating kinematics...

        Trial object saved as ../tracking/full/SF_Sat_14dpf_GroupA_n2a_2020-06-13-090438-0000/trial.pik 

       ... kinematics calculated for Trial and saved in
             ../tracking/full/SF_Sat_14dpf_GroupA_n2a_2020-06-13-090438-0000/trial.pik 


  Generating occlusion cut using min d_nn = 0.00 cm with 2 buffer frames
 Shape before cuts:  (

<Figure size 432x288 with 0 Axes>